In [ ]:
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.core import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from math import floor
from imageio import imread
from skimage.transform import resize
import numpy as np
from numpy.random import randint
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from os.path import join as pJoin
from os import listdir

In [ ]:
NewSize = (160, 320)
ModelDir = '/media/anubis/Data/OpenVINO/UnetFull/LAST'
SegModelName = '0A_100E_160x320D.h5'
ClassModelName = 'Class_100E_160x320D.h5'
SegModelFile = pJoin(ModelDir, SegModelName)

In [ ]:
def UNetFull(img_rows, img_cols, img_channels):  # 23 trainable layers, use same padding, en lugar de unpadding layers
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    # Encoder 
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv5)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)
    
    conv7 = Conv2D(512, (3, 3), padding='same', activation='relu')(pool3)
    conv8 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv7)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv8)
    
    conv9 = Conv2D(1024, (3, 3), padding='same', activation='relu')(pool4)
    conv10 = Conv2D(1024, (3, 3), padding='same', activation='relu')(conv9)
    
    # Decoder
    convT1 = Conv2D(512, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv10))
    merge1 = concatenate([convT1, conv8], axis=3)
    conv11 = Conv2D(512, (3, 3), padding='same', activation='relu')(merge1)
    conv12 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv11)
    
    convT2 = Conv2D(256, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv12))
    merge2 = concatenate([convT2, conv6], axis=3)
    conv13 = Conv2D(256, (3, 3), padding='same', activation='relu')(merge2)
    conv14 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv13)
    
    convT3 = Conv2D(128, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv14))
    merge3 = concatenate([convT3, conv4], axis=3)
    conv15 = Conv2D(128, (3, 3), padding='same', activation='relu')(merge3)
    conv16 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv15)
    
    convT4 = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv16))
    merge4 = concatenate([convT4, conv2], axis=3)
    conv17 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge4)
    conv18 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv17)

    y = Conv2D(2, (1, 1), activation='softmax')(conv18)

    return Model(inputs = x, outputs = y)

In [ ]:
model = UNetFull(NewSize[0], NewSize[1], 3)
model.load_weights(SegModelFile)

In [ ]:
x = model.layers[-24].output
predictions = Flatten()(x)
predictions = Dense(1024, activation = 'relu')(predictions)
predictions = Dropout(0.5)(predictions)
predictions = Dense(512, activation = 'relu')(predictions)
predictions = Dropout(0.5)(predictions)
predictions = Dense(3, activation = 'softmax')(predictions)
classifier = Model(inputs = model.inputs, outputs=predictions)

for layer in classifier.layers[:-6]:
        layer.trainable = False
            
print(classifier.summary())

In [ ]:
DATA_DIR = '/media/anubis/Data/OpenVINO/Classification'
TRAIN_DIR = pJoin(DATA_DIR, "train")
VAL_DIR = pJoin(DATA_DIR, "validation")
BATCH_SIZE = 10

train_datagen =  ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size = NewSize,
                                                    class_mode = 'categorical',
                                                    batch_size = BATCH_SIZE,
                                                   )

validation_generator = test_datagen.flow_from_directory(VAL_DIR,
                                                        target_size = NewSize, 
                                                        batch_size = floor(BATCH_SIZE),
                                                        class_mode = 'categorical'
                                                        )

In [ ]:
NUM_EPOCHS = 100

optimizer = SGD(lr = 0.009)

classifier.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint(pJoin(ModelDir, ClassModelName), monitor='val_loss',  verbose=0, save_best_only=True,
                             mode='min', period=1)

In [ ]:
history = classifier.fit_generator(train_generator, epochs = NUM_EPOCHS,  
                                       steps_per_epoch = len(train_generator), 
                                       shuffle = True,
                                       callbacks = [checkpoint],
                                       validation_data = validation_generator,
                                       validation_steps = len(validation_generator))

plt.rcParams['figure.figsize'] = (12, 4) # Hacer las figuras más grandes

plt.subplot(1, 2, 1)
BestAcc = np.argmax(history.history['val_acc'])
plt.plot(history.history['acc'], label='Training')
plt.plot(history.history['val_acc'], label='Validation')
plt.plot(BestAcc, history.history['acc'][BestAcc],'go')
plt.plot(BestAcc, history.history['val_acc'][BestAcc],'go')
plt.ylabel('Accuracy', fontsize = 20)
plt.xlabel('Epoch', fontsize = 20)
plt.suptitle('Learning curves', fontsize = 28)
plt.subplot(1, 2, 2)
BestLoss = np.argmin(history.history['val_loss'])
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.plot(BestLoss, history.history['val_loss'][BestLoss],'go')
plt.plot(BestLoss, history.history['loss'][BestLoss],'go')
plt.xlabel('Epoch', fontsize = 20)
plt.ylabel('Loss', fontsize = 20)

plt.savefig(pJoin(ModelDir, 'ACC_LOSS_' + ClassModelName + '.png'), bbox_inches='tight')
plt.show()

In [ ]:
classifier.load_weights(pJoin(ModelDir, 'Classifier.h5'))
TrainScore = classifier.evaluate_generator(train_generator, steps=89)
ValidationScore = classifier.evaluate_generator(validation_generator, steps=89)
file = open(pJoin(ModelDir, 'BestClassifierAssess_' + ClassModelName + '.txt'), 'w') 
file.write('Training\n')
for Value in TrainScore:
    file.write(str(Value) + '\n') 
file.write('Validation\n')
for Value in ValidationScore:
    file.write(str(Value) + '\n')
file.close()
print('Model assess saved')

In [ ]:
import pandas as pd

hist_df = pd.DataFrame(history.history) 

hist_csv_file = pJoin(ModelDir, 'ClassifierHistory_' + ClassModelName + '.csv')
with open(hist_csv_file, mode = 'w') as f:
    hist_df.to_csv(f)

In [ ]:
img_class = randint(0, 3)
img_dir = pJoin(DATA_DIR, str(img_class))
img_names = listdir(img_dir)
file_name = img_names[randint(0, len(img_names))]

image_file1 = pJoin(img_dir, file_name)

img_original = imread(image_file1)
img = resize(img_original, NewSize, mode = 'constant')

img1 = np.expand_dims(img, axis=0)
img1.shape

response = classifier.predict(img1)
print(response)

imshow(img)
plt.title(file_name.split('.')[0] + '. Class: ' + str(img_class) + ' Diagnosis: ' + str(response.argmax()))